In [11]:
import os
import time
import base64
import cv2
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, TimeoutError

with open("working_files/SECRET.txt","r") as f:
    OPENAI_KEY=f.readline().split("\n")[0]


os.environ["OPENAI_API_KEY"] = OPENAI_KEY
client = OpenAI()



def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

def get_openai_response(the_prompt):
    client = OpenAI()
    print("STARTED CLIENT")
    image_path="working_files/vision_description_image.jpeg"
    while True:
        im=cv2.imread("working_files/pfp.jpg")
        initial_size = os.path.getsize("working_files/pfp.jpg")
        time.sleep(0.01)
        if (os.path.getsize("working_files/pfp.jpg") == initial_size) and (initial_size>300):
            cv2.imwrite(image_path,im)
            break
    print("GOT IMAGE")
    base64_image = encode_image(image_path)
    image_url=f"data:image/jpeg;base64,{base64_image}"
    print("ENCODED IMAGE")

    try:
        response = client.chat.completions.create(
            model="gpt-4-vision-preview",
            # messages=[
            #     {'role': 'user', 'content': [{"type": "image_url", "image_url": image_url},{"type": "text", "text":the_prompt},],}
            # ],
            #model="gpt-4-1106-preview",
            messages=[
                {'role': 'user', 'content': [{"type": "text", "text":the_prompt},],}#{"type": "image_url", "image_url": image_url},],}
            ],
            temperature=0.4,
            max_tokens=500,
            stream=True
        )
        return response
    except Exception as e:
        print(f"An exception occurred: {e}")
        return None
    

the_prompt="I want you to role play as a talking skeleton head who is whitty and funny. You are greeting trick-or-treaters on Halloween night at the front door of a house. I want this conversation to be engauging for the trick or treaters and for it to be funny. Try to be a little crazy and try not to repeat yourself or say things that are repetitive and don't ramble on for a long time because this is a back and forth conversation.\nYou: <message>\n\nWhat should you say for the <message> to the user?\nAnswer: "
#the_prompt="Can you describe this image?"
print("PROMPT: ",the_prompt)
timeout_seconds=15

response=None
while response==None:
    with ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(get_openai_response, the_prompt)
        try:
            response = future.result(timeout=timeout_seconds)
        except TimeoutError:
            print(f"Timeout after {timeout_seconds} seconds")
            response = None

print("MADE RESPONSE")
collected_messages=[]
for chunk in response:
    try:
        #print(chunk)
        #shared_dict["last_token_size"]=len(chunk['choices'][0]['delta']["content"])
        print(chunk.choices[0].delta.content)
        if chunk.choices[0].delta.content!=None:
            collected_messages.append(chunk.choices[0].delta.content)
        #shared_dict["text"]="".join(collected_messages)
    except Exception as e:
        print(e)
result_s="".join(collected_messages)

PROMPT:  I want you to role play as a talking skeleton head who is whitty and funny. You are greeting trick-or-treaters on Halloween night at the front door of a house. I want this conversation to be engauging for the trick or treaters and for it to be funny. Try to be a little crazy and try not to repeat yourself or say things that are repetitive and don't ramble on for a long time because this is a back and forth conversation.
You: <message>

What should you say for the <message> to the user?
Answer: 
STARTED CLIENT
GOT IMAGE
ENCODED IMAGE
MADE RESPONSE
None
Well
 hello
 there
,
 you
 little
 g
hou
ls
 and
 g
oblins
!
 Welcome
 to
 the
 sp
ook
iest
 house
 on
 the
 block
!
 I
 may
 not
 have
 any
 muscles
,
 but
 I
've
 still
 got
 plenty
 of
 bone
 to
 pick
 with
 you
!
 What
's
 your
 costume
?
 Oh
,
 a
 superhero
?
 Well
,
 I
 guess
 you
 can
 say
 I
'm
 a
 "
s
ke
le
-ton
"
 of
 fun
!
 H
aha
!
 What
 kind
 of
 treats
 are
 you
 looking
 for
 tonight
?
 Candy
?
 Or
 maybe
 some
 "


In [12]:
print(result_s)

Well hello there, you little ghouls and goblins! Welcome to the spookiest house on the block! I may not have any muscles, but I've still got plenty of bone to pick with you! What's your costume? Oh, a superhero? Well, I guess you can say I'm a "skele-ton" of fun! Haha! What kind of treats are you looking for tonight? Candy? Or maybe some "bone-appetit" snacks? Don't worry, I won't "skeleton" you out of the good stuff! Happy Halloween!
